# Tile Server Process for Aurora Outputs

In [1]:
import os
import sys

sys.path.append('../')

print(os.getcwd())
print(os.listdir())

import yaml
from PIL import Image
import numpy as np
import torch
import xarray as xr
import matplotlib.pyplot as plt
import cfgrib
from datetime import datetime
from io import BytesIO
import ast
from cartopy import crs as ccrs, feature as cfeature
from tile_server import save_tile, tile_into_folders

/home/jgib124/aurora/aurora/afwerx
['tile_server_process.ipynb', '__pycache__', 'aurora_regridding_process.ipynb', 'visualize.py', 'configs', 'tile_server.py', 'serve', 'tiles', 'aurora_inference_simple.py']


/home/jgib124/miniconda3/envs/aurora_test/lib/python3.11/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


Wasabi: 
	Endpoint:  https://s3.wasabisys.com/
	Bucket:  aa-archiver
IDrive: 
	Endpoint:  https://l5l2.va.idrivee2-56.com
	Bucket:  archive-mosaics


In [2]:
# Read in regridded data
ds = xr.open_dataset("tiles/aurora_predictions_regridded.nc", engine='netcdf4')

ds.variables.keys()

KeysView(Frozen({'surf_2t': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'surf_10u': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'surf_10v': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'surf_msl': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'static_z': <xarray.Variable (y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'static_slt': <xarray.Variable (y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'static_lsm': <xarray.Variable (y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'atmos_t': <xarray.Variable (batch: 1, history: 1, level: 13, y: 4608, x: 7168)> Size: 2GB
[429391872 values with dtype=float32], 'atmos_u': <xarray.Variable (batch: 1, history: 1, level: 13, y: 4608

In [3]:
serve_dir = "serve"
noun = "notebook_test"
timestamp = "20230101"
var = 'atmos_t'

selected_array = ds[var].isel(batch=0, level=0, history=0).to_numpy()

# lat, lon, channels
selected_array.shape

(4608, 7168)

In [ ]:
# NOTE: either 1D or 3D image
# TODO: check for number of channels and repeat if 1D
serve_dir = "serve"
noun = "notebook_test"
timestamp = "20230101"

for var in ['atmos_t', 'atmos_q', 'atmos_v']:
    selected_array = ds[var].isel(batch=0, level=0, history=0).to_numpy()
    tile_into_folders(serve_dir=serve_dir, noun=var, timestamp=timestamp, image=selected_array)

Image Tile:  (256, 256, 3)
File Path:  serve/notebook_test/20230101/7/0/0.png
Input Dimensions:  (256, 256)
After Colorization:  (256, 256, 4)
Tile Shape:  (256, 256, 4)
Saving image to:  serve/notebook_test/20230101/7/0/0.png
serve/notebook_test/20230101/7/0/0.png Status:  True
Image Tile:  (256, 256, 3)
File Path:  serve/notebook_test/20230101/7/0/1.png
Input Dimensions:  (256, 256)
After Colorization:  (256, 256, 4)
Tile Shape:  (256, 256, 4)
Saving image to:  serve/notebook_test/20230101/7/0/1.png
serve/notebook_test/20230101/7/0/1.png Status:  True
Image Tile:  (256, 256, 3)
File Path:  serve/notebook_test/20230101/7/0/2.png
Input Dimensions:  (256, 256)
After Colorization:  (256, 256, 4)
Tile Shape:  (256, 256, 4)
Saving image to:  serve/notebook_test/20230101/7/0/2.png
serve/notebook_test/20230101/7/0/2.png Status:  True
Image Tile:  (256, 256, 3)
File Path:  serve/notebook_test/20230101/7/1/0.png
Input Dimensions:  (256, 256)
After Colorization:  (256, 256, 4)
Tile Shape:  (256

[ WARN:0@1.125] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
